In [1]:
import cantera as ct
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import cm
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
plt.switch_backend('agg')  # needed for saving figures
import csv
import os
# import rmgpy
# import rmg
import re
import operator
import pandas as pd
import pylab
from cycler import cycler
import seaborn as sns
import os
import multiprocessing
from functools import partial
import threading
import itertools
from rmgpy.molecule import Molecule
from rmgpy.data.base import Database
import subprocess

In [2]:
mm = 0.001
cm = 0.01
ms = mm
minute = 60.0

#######################################################################
# Input Parameters
#######################################################################
t_in = 800  # K - in the paper, it was ~698.15K at the start of the cat surface and ~373.15 for the gas inlet temp
t_cat = t_in
length = 70 * mm  # Reactor length - m
diam = 16.5 * mm  # Reactor diameter - in m, from Ref 17 & Ref 18
area = (diam/2.0)**2*np.pi  # Reactor cross section area (area of tube) in m^2
porosity = 0.81  # Monolith channel porosity, from Ref 17, sec 2.2.2
cat_area_per_vol = 1.6e4  # m2/m3, which is 160 cm2/cm3, as used in Horn 2006
cat_area_per_vol *= 0.05 # to make the concentrations change slower
flow_rate = 4.7  # slpm, as seen in as seen in Horn 2007
tot_flow = 0.208  # constant inlet flow rate in mol/min, equivalent to 4.7 slpm
flow_rate = flow_rate * .001 / 60  # m^3/s, as seen in as seen in Horn 2007
velocity = flow_rate / area  # m/s
MOLECULAR_WEIGHTS = {"H": 1.008, "O": 15.999, "C": 12.011, "N": 14.0067}

# The PFR will be simulated by a chain of 'N_reactors' stirred reactors.
N_reactors = 7001

on_catalyst = 1000  # catalyst length 10mm, from Ref 17
off_catalyst = 2000
dt = 1.0

# new sensitivities
# length = 510 * mm  # Reactor length - m
# N_reactors = 51001
# on_catalyst = 1000  # catalyst length 10mm, from Ref 17
# off_catalyst = 51000

reactor_len = length/(N_reactors-1)
rvol = area * reactor_len * porosity

# catalyst area in one reactor
cat_area = cat_area_per_vol * rvol

# root directory for output files
out_root = os.getcwd()

residence_time = reactor_len / velocity # unit in s

def setup_ct_solution(path_to_cti):

    # this chemkin file is from the cti generated by rmg
    gas = ct.Solution(path_to_cti, 'gas')
    surf = ct.Interface(path_to_cti, 'surface1', [gas])

    print("This mechanism contains {} gas reactions and {} surface reactions".format(gas.n_reactions, surf.n_reactions))
    print(f"Thread ID from threading{threading.get_ident()}")
    i_ar = gas.species_index('Ar')


    return {'gas':gas, 'surf':surf,"i_ar":i_ar,"n_surf_reactions":surf.n_reactions}

def transient_flux_diagram(reaction_index, surf, out_dir, ratio):
    """
    Plot the transient flux diagram at a certain location
    """
    location = str(int(reaction_index / 100))
    diagram = ct.ReactionPathDiagram(surf, 'X')
    diagram.title = 'rxn path'
    diagram.label_threshold = 1e-9
    dot_file = f"{out_dir}/rxnpath-{ratio:.1f}-x-{location}mm.dot"
    img_file = f"{out_dir}/rxnpath-{ratio:.1f}-x-{location}mm.pdf"
    diagram.write_dot(dot_file)
    os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))

    for element in elements:
        diagram = ct.ReactionPathDiagram(surf, element)
        diagram.title = element + 'rxn path'
        diagram.label_threshold = 1e-9
        dot_file = f"{out_dir}/rxnpath-{ratio:.1f}-x-{location}mm-{element}.dot"
        img_file = f"{out_dir}/rxnpath-{ratio:.1f}-x-{location}mm-{element}.pdf"
        diagram.write_dot(dot_file)
        os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))

def add_fluxes(gas, surf, total_fluxes):
    """
    Add the current fluxes to the stored totals.
    """
    gas_fluxes = total_fluxes["gas"]
    surf_fluxes = total_fluxes["surf"]
    for element in "HOC":
        try:
            gas_fluxes[element].add(ct.ReactionPathDiagram(gas, element))
        except KeyError:
            gas_fluxes[element] = ct.ReactionPathDiagram(gas, element)
        try:
            surf_fluxes[element].add(ct.ReactionPathDiagram(surf, element))
        except KeyError:
            surf_fluxes[element] = ct.ReactionPathDiagram(surf, element)
    # Now do the 'X' for just the surface
    element = "X"
    try:
        surf_fluxes[element].add(ct.ReactionPathDiagram(surf, element))
    except KeyError:
        surf_fluxes[element] = ct.ReactionPathDiagram(surf, element)
    return gas_fluxes, surf_fluxes

def prettydot(dotfilepath, strip_line_labels=False):
    """
    Make a prettier version of the dot file (flux diagram)

    Assumes the species pictures are stored in a directory
    called 'species_pictures' alongside the dot file.
    """

    # pictures_directory = os.path.join(os.path.split(dotfilepath)[0], "species_pictures")
    pictures_directory = "/work/westgroup/chao/sketches/cpox_sim/rmg_models/base_cathub/species"

    if strip_line_labels:
        print("stripping edge (line) labels")

    # replace this:
    #  s10 [ fontname="Helvetica", label="C11H23J"];
    # with this:
    #  s10 [ shapefile="mols/C11H23J.png" label="" width="1" height="1" imagescale=true fixedsize=true color="white" ];

    reSize = re.compile('size="5,6"\;page="5,6"')
    reNode = re.compile(
        '(?P<node>s\d+)\ \[\ fontname="Helvetica",\ label="(?P<label>[^"]*)"\]\;'
    )

    rePicture = re.compile("(?P<smiles>.+?)\((?P<id>\d+)\)\.png")
    reLabel = re.compile("(?P<name>.+?)\((?P<id>\d+)\)$")

    species_pictures = dict()
    for picturefile in os.listdir(pictures_directory):
        match = rePicture.match(picturefile)
        if match:
            species_pictures[match.group("id")] = picturefile
        else:
            pass
            # print(picturefile, "didn't look like a picture")

    filepath = dotfilepath

    if not open(filepath).readline().startswith("digraph"):
        raise ValueError("{0} - not a digraph".format(filepath))

    infile = open(filepath)
    prettypath = filepath + "-pretty"
    outfile = open(prettypath, "w")

    for line in infile:
        (line, changed_size) = reSize.subn('size="12,12";page="12,12"', line)
        match = reNode.search(line)
        if match:
            label = match.group("label")
            idmatch = reLabel.match(label)
            if idmatch:
                idnumber = idmatch.group("id")
                if idnumber in species_pictures:
                    line = (
                        '%s [ image="%s/%s" label="" width="0.5" height="0.5" imagescale=false fixedsize=false color="none" ];\n'
                        % (match.group("node"),pictures_directory, species_pictures[idnumber])
                    )

        # rankdir="LR" to make graph go left>right instead of top>bottom

        if strip_line_labels:
            line = re.sub('label\s*=\s*"\s*[\d.]+"', 'label=""', line)

        # change colours
        line = re.sub('color="0.7,\ (.*?),\ 0.9"', r'color="1.0, \1, 0.7*\1"', line)

        outfile.write(line)

    outfile.close()
    infile.close()
    print(f"Graph saved to: {prettypath}")
    return prettypath

def get_current_fluxes(gas, surf):
    """
    Get all the current fluxes.
    Returns a dict like:
        `fluxes['gas']['C'] = ct.ReactionPathDiagram(self.gas, 'C')` etc.
    """
    fluxes = {"gas": dict(), "surf": dict()}
    for element in "HOC":
        fluxes["gas"][element] = ct.ReactionPathDiagram(gas, element)
        fluxes["surf"][element] = ct.ReactionPathDiagram(surf, element)
    element = "X"  # just the surface
    fluxes["surf"][element] = ct.ReactionPathDiagram(surf, element)
    return fluxes

def combine_fluxes(fluxes_dict):
    """
    Combined a dict of dicts of flux diagrams into one.

    Fluxes should be a dict with entries like
        fluxes['gas']['C'] = ct.ReactionPathDiagram(self.gas, 'C')

    Returns the flux diagram a string in the format you'd get from
    ct.ReactionPathdiagram.get_data()
    """
    # getting the entire net rates of the system
    temp_flux_data = dict()
    species = set()
    for element in "HOC":
        for phase in ("gas", "surf"):
            data = fluxes_dict[phase][element].get_data().strip().splitlines()
            if not data:
                # eg. if there's no gas-phase reactions involving C
                continue
            species.update(data[0].split())  # First line is a list of species
            for line in data[1:]:  # skip the first line

                s1, s2, fwd, rev = line.split()
                these_fluxes = np.array([float(fwd), float(rev)])

                if all(these_fluxes == 0):
                    continue

                # multiply by atomic mass of the element
                these_fluxes *= MOLECULAR_WEIGHTS[element]

                # for surface reactions, multiply by the catalyst area per volume in a reactor
                if phase == "surf":
                    these_fluxes *= cat_area_per_vol

                try:
                    # Try adding in this direction
                    temp_flux_data[(s1, s2)] += these_fluxes
                except KeyError:
                    try:
                        # Try adding in reverse direction
                        temp_flux_data[(s2, s1)] -= these_fluxes
                    except KeyError:
                        # Neither direction there yet, so create in this direction
                        temp_flux_data[(s1, s2)] = these_fluxes

    output = " ".join(species) + "\n"
    output += "\n".join(
        f"{s1} {s2} {fwd} {rev}" for (s1, s2), (fwd, rev) in temp_flux_data.items()
    )
    return output

def write_flux_dot(flux_data_string, out_file_path, threshold=0.01, title=""):
    """
    Takes a flux data string fromatted as from ct.ReactionPathdiagram.get_data()
    (or from combine_fluxes) and makes a graphviz .dot file.

    Fluxes below 'threshold' are not plotted.
    """

    output = ["digraph reaction_paths {", "center=1;"]

    flux_data = {}
    species_dict = {}
    flux_data_lines = flux_data_string.splitlines()
    species = flux_data_lines[0].split()

    for line in flux_data_lines[1:]:
        s1, s2, fwd, rev = line.split()
        net = float(fwd) + float(rev)
        if net < 0.0:  # if net is negative, switch s1 and s2 so it is positive
            flux_data[(s2, s1)] = -1 * net
        else:
            flux_data[(s1, s2)] = net

        # renaming species to dot compatible names
        if s1 not in species_dict:
            species_dict[s1] = "s" + str(len(species_dict) + 1)
        if s2 not in species_dict:
            species_dict[s2] = "s" + str(len(species_dict) + 1)

    # getting the arrow widths
    largest_rate = max(flux_data.values())

    added_species = {}  # dictionary of species that show up on the diagram
    for (s1, s2), net in flux_data.items():  # writing the node connections

        flux_ratio = net / largest_rate
        if abs(flux_ratio) < threshold:
            continue  # don't include the paths that are below the threshold

        pen_width = (
            1.0 - 4.0 * np.log10(flux_ratio / threshold) / np.log10(threshold) + 1.0
        )
        # pen_width = ((net - smallest_rate) / (largest_rate - smallest_rate)) * 4 + 2
        arrow_size = min(6.0, 0.5 * pen_width)
        output.append(
            f'{species_dict[s1]} -> {species_dict[s2]} [fontname="Helvetica", penwidth={pen_width:.2f}, arrowsize={arrow_size:.2f}, color="0.7, {flux_ratio+0.5:.3f}, 0.9", label="{flux_ratio:0.3g}"];'
        )

        added_species[s1] = species_dict[s1]
        added_species[s2] = species_dict[s2]

    for (
        species,
        s_index,
    ) in added_species.items():  # writing the species translations
        output.append(f'{s_index} [ fontname="Helvetica", label="{species}"];')

    title_string = (r"\l " + title) if title else ""
    output.append(f' label = "Scale = {largest_rate}{title_string}";')
    output.append(' fontname = "Helvetica";')
    output.append("}\n")

    directory = os.path.split(out_file_path)[0]
    if directory:
        os.makedirs(directory, exist_ok=True)
    with open(out_file_path, "w") as out_file:
        out_file.write("\n".join(output))
    return "\n".join(output)


def save_flux_diagrams(gas, surf, total_fluxes, path="", suffix="", fmt="png"):
    """
    Saves the flux diagrams, in the provided path.
    The filenames have a suffix if provided,
    so you can keep them separate and not over-write.
    fmt can be 'pdf' or 'png'
    """
    # for element in "CHOX":
    #     for phase_object in (gas, surf):
    #         phase = phase_object.name

    #         diagram = ct.ReactionPathDiagram(phase_object, element)
    #         diagram.title = f"Reaction path diagram following {element} in {phase}"
    #         diagram.label_threshold = 0.01

    #         dot_file = os.path.join(
    #             path,
    #             f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.dot",
    #         )
    #         img_file = os.path.join(
    #             path,
    #             f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.{fmt}",
    #         )
    #         diagram.write_dot(dot_file)
    #         # print(diagram.get_data())

    #         print(
    #             f"Wrote graphviz input file to '{os.path.join(os.getcwd(), dot_file)}'."
    #         )

    #         # Unufortunately this code is duplicated below,
    #         # so be sure to duplicate any changes you make!
    #         pretty_dot_file = prettydot(dot_file)
    #         subprocess.run(
    #             [
    #                 "dot",
    #                 os.path.abspath(pretty_dot_file),
    #                 f"-T{fmt}",
    #                 "-o",
    #                 os.path.abspath(img_file),
    #                 "-Gdpi=72",
    #             ],
    #             cwd=path,
    #             check=True,
    #         )
    #         print(
    #             f"Wrote graphviz output file to '{os.path.join(os.getcwd(), img_file)}'."
    #         )
    strings = []
    # Now do the combined flux
    for name, fluxes_dict in [
        ("mass", get_current_fluxes(gas, surf)),
        ("integrated mass", total_fluxes),
    ]:

        flux_data_string = combine_fluxes(fluxes_dict)
        strings.append(flux_data_string)
        dot_file = os.path.join(
            path,
            f"reaction_path_{name.replace(' ','_')}{'_' if suffix else ''}{suffix}_total.dot",
        )
        img_file = os.path.join(
            path,
            f"reaction_path_{name.replace(' ','_')}{'_' if suffix else ''}{suffix}_total.{fmt}",
        )
        write_flux_dot(
            flux_data_string,
            dot_file,
            threshold=0.01,
            title=f"Reaction path diagram showing combined {name}",
        )
        # Unufortunately this code is duplicated above,
        # so be sure to duplicate any changes you make!
        pretty_dot_file = prettydot(dot_file)

        subprocess.run(
            [
                "dot",
                os.path.abspath(pretty_dot_file),
                f"-T{fmt}",
                "-o",
                os.path.abspath(img_file),
                "-Gdpi=72",
            ],
            cwd=path,
            check=True,
        )
        print(
            f"Wrote graphviz output file to '{os.path.abspath(os.path.join(os.getcwd(), img_file))}'."
        )


def monolith_simulation(path_to_cti, temp, mol_in, rtol, atol, verbose=False, sens=False):
    """
    Set up and solve the monolith reactor simulation.
    Verbose prints out values as you go along
    Sens is for sensitivity, in the form [perturbation, reaction #]
    Args:
        path_to_cti: full path to the cti file
        temp (float): The temperature in Kelvin
        mol_in (3-tuple or iterable): the inlet molar ratios of (CH4, O2, He)
        verbose (Boolean): whether to print intermediate results
        sens (False or 2-tuple/list): if not False, then should be a 2-tuple or list [dk, rxn]
                in which dk = relative change (eg. 0.01) and rxn = the index of the surface reaction rate to change
    Returns:
        gas_out, # gas molar flow rate in moles/minute
        surf_out, # surface mole fractions
        gas_names, # gas species names
        surf_names, # surface species names
        dist_array, # distances (in mm)
        T_array # temperatures (in K)
    """
    sols_dict = setup_ct_solution(path_to_cti)
    gas, surf, i_ar, n_surf_reactions= sols_dict['gas'], sols_dict['surf'], sols_dict['i_ar'],sols_dict['n_surf_reactions']
    print(f"Running monolith simulation with CH4 and O2 concs {mol_in[0], mol_in[1]} on thread {threading.get_ident()}")
    ch4, o2, ar = mol_in
    ratio = ch4 / (2 * o2)

    X = f"CH4(2):{ch4}, O2(3):{o2}, Ar:{ar}"
    gas.TPX = 273.15, ct.one_atm, X  # need to initialize mass flow rate at STP
    # mass_flow_rate = velocity * gas.density_mass * area  # kg/s
    mass_flow_rate = flow_rate * gas.density_mass
    gas.TPX = temp, ct.one_atm, X
    temp_cat = temp
    surf.TP = temp_cat, ct.one_atm
    surf.coverages = 'X(1):1.0'
    gas.set_multiplier(1.0)

    TDY = gas.TDY
    cov = surf.coverages

    if verbose is True:
        print('  distance(mm)   X_CH4        X_O2        X_H2       X_CO       X_H2O       X_CO2')

    # create a new reactor
    gas.TDY = TDY
    r = ct.IdealGasReactor(gas)
    r.volume = rvol

    # create a reservoir to represent the reactor immediately upstream. Note
    # that the gas object is set already to the state of the upstream reactor
    upstream = ct.Reservoir(gas, name='upstream')

    # create a reservoir for the reactor to exhaust into. The composition of
    # this reservoir is irrelevant.
    downstream = ct.Reservoir(gas, name='downstream')

    # Add the reacting surface to the reactor. The area is set to the desired
    # catalyst area in the reactor.
    rsurf = ct.ReactorSurface(surf, r, A=cat_area)

    # The mass flow rate into the reactor will be fixed by using a
    # MassFlowController object.
    # mass_flow_rate = velocity * gas.density_mass * area  # kg/s
    # mass_flow_rate = flow_rate * gas.density_mass
    m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

    # We need an outlet to the downstream reservoir. This will determine the
    # pressure in the reactor. The value of K will only affect the transient
    # pressure difference.
    v = ct.PressureController(r, downstream, master=m, K=1e-5)

    sim = ct.ReactorNet([r])
    sim.max_err_test_fails = 12

    # set relative and absolute tolerances on the simulation
    sim.rtol = rtol
    sim.atol = atol

    gas_names = gas.species_names
    surf_names = surf.species_names
    gas_out = []
    surf_out = []
    dist_array = []
    T_array = []
    net_rates_of_progress = []
    
    total_flux = {"gas": dict(), "surf": dict()}
    surf_diagrams = []
    rsurf.kinetics.set_multiplier(0.0)  # no surface reactions until the gauze
    for n in range(N_reactors):
        # Set the state of the reservoir to match that of the previous reactor
        gas.TDY = r.thermo.TDY
        upstream.syncState()
        if n == on_catalyst:
            rsurf.kinetics.set_multiplier(1.0)
            if sens is not False:
                rsurf.kinetics.set_multiplier(1.0 + sens[0], sens[1])
        if n == off_catalyst:
            rsurf.kinetics.set_multiplier(0.0)
        sim.reinitialize()
        sim.advance(sim.time + 1e4 * residence_time)
        dist = n * reactor_len * 1.0e3  # distance in mm
        dist_array.append(dist)
        T_array.append(rsurf.kinetics.T)
        kmole_flow_rate = mass_flow_rate / gas.mean_molecular_weight  # kmol/s
        gas_out.append(1000 * 60 * kmole_flow_rate * r.kinetics.X.copy())  # molar flow rate in moles/minute
        surf_out.append(rsurf.kinetics.X.copy())
        net_rates_of_progress.append(rsurf.kinetics.net_rates_of_progress)
        # surfs.append(surf)
        # stop simulation when things are done changing, to avoid getting so many COVDES errors
        if n >= 1001:
            if np.max(abs(np.subtract(gas_out[-2], gas_out[-1]))) < 1e-15:
                break
        if n>0:
            # total_flux['gas'], total_flux['surf'] = add_fluxes(gas, surf, total_flux)
            # gas_fluxes = total_flux["gas"]
            # surf_fluxes = total_flux["surf"]
            for element in "HOC":
                try:
                    gas_fluxes[element].add(ct.ReactionPathDiagram(gas, element))
                except KeyError:
                    gas_fluxes[element] = ct.ReactionPathDiagram(gas, element)
                try:
                    surf_fluxes[element].add(ct.ReactionPathDiagram(surf, element))
                except KeyError:
                    surf_fluxes[element] = ct.ReactionPathDiagram(surf, element)
            # Now do the 'X' for just the surface
            element = "X"
            try:
                surf_fluxes[element].add(ct.ReactionPathDiagram(surf, element))
                # if n > 1000:
                #     with open('surf_log.txt', 'a') as f:
                #         f.write(surf_fluxes[element].get_data())
                #     with open('surf_log2.txt', 'a') as f:
                #         f.write(ct.ReactionPathDiagram(surf, element).get_data())
            except KeyError:
                surf_fluxes[element] = ct.ReactionPathDiagram(surf, element)
                surf_diagrams.append(ct.ReactionPathDiagram(surf, element))

        # make reaction diagrams
        out_dir = 'rxnpath'
        os.path.exists(out_dir) or os.makedirs(out_dir)
        elements = ['H', 'O', 'C']
        locations_of_interest = [1045]
        if sens is False:
            if n in locations_of_interest:
                # # plot the transient flux_diagrams
                # transient_flux_diagram(n, surf, out_dir, ratio)
                # save_flux_diagrams(gas, surf, total_flux, path='rxnpath', suffix=f'{ratio:.1f}_{n}')
                df = pd.DataFrame(surf.net_rates_of_progress)
                df.to_csv(f"{out_dir}/net_rates_{ratio:.1f}_{n}.csv")
            # if (not (n-1) % 100) or n == (self.number_of_reactors - 1):
            #     save_flux_data(
            #         f"flux_data_Temp_{r.T-273.15:.0f}C_Dist_{distance_mm:.1f}"
            #     )

        if verbose is True:
            if not n % 100:
                print('  {0:10f}  {1:10f}  {2:10f}  {3:10f} {4:10f} {5:10f} {6:10f}'.format(dist, *gas[
                    'CH4(2)', 'O2(3)', 'H2(6)', 'CO(7)', 'H2O(5)', 'CO2(4)'].X * 1000 * 60 * kmole_flow_rate))

    gas_out = np.array(gas_out)
    surf_out = np.array(surf_out)
    gas_names = np.array(gas_names)
    surf_names = np.array(surf_names)
    data_out = gas_out, surf_out, gas_names, surf_names, dist_array, T_array, i_ar, n_surf_reactions
    
    net_rates_of_progress = np.array(net_rates_of_progress)
    df = pd.DataFrame(net_rates_of_progress)
    
    out_dir = os.path.join(out_root, 'rates_of_progress')
    os.path.exists(out_dir) or os.makedirs(out_dir)
    df.to_csv(f"{out_dir}/net_rates_{ratio:.1f}.csv")
    print(len(dist_array))
    print(f"Finished monolith simulation for CH4 and O2 concs {mol_in[0], mol_in[1]} on thread {threading.get_ident()}")
    return data_out



In [3]:
import warnings
warnings.filterwarnings('ignore')
import copy

surf_diagrams_total = []
surf_diagrams_current = []
ratio = 0.6
plot_generated = False
fo2 = 1 / (2. * ratio + 1 + 79. / 21.)
fch4 = 2 * fo2 * ratio
far = 79 * fo2 / 21
ratio_in = [fch4, fo2, far]  # mol fractions

# a = monolith_simulation('cantera.yaml', t_in, ratio_in, 1e-16, 1e-8)
# gas_out, surf_out, gas_names, surf_names, dist_array, T_array, i_ar, n_surf_reactions = a 
path_to_cti = 'cantera.yaml'
atol = 1e-16
rtol = 1e-8
mol_in = ratio_in
verbose=False
sens=False
temp=t_in

sols_dict = setup_ct_solution(path_to_cti)
gas, surf, i_ar, n_surf_reactions= sols_dict['gas'], sols_dict['surf'], sols_dict['i_ar'],sols_dict['n_surf_reactions']
print(f"Running monolith simulation with CH4 and O2 concs {mol_in[0], mol_in[1]} on thread {threading.get_ident()}")
ch4, o2, ar = mol_in
ratio = ch4 / (2 * o2)

X = f"CH4(2):{ch4}, O2(3):{o2}, Ar:{ar}"
gas.TPX = 273.15, ct.one_atm, X  # need to initialize mass flow rate at STP
# mass_flow_rate = velocity * gas.density_mass * area  # kg/s
mass_flow_rate = flow_rate * gas.density_mass
gas.TPX = temp, ct.one_atm, X
temp_cat = temp
surf.TP = temp_cat, ct.one_atm
surf.coverages = 'X(1):1.0'
gas.set_multiplier(1.0)

TDY = gas.TDY
cov = surf.coverages

if verbose is True:
    print('  distance(mm)   X_CH4        X_O2        X_H2       X_CO       X_H2O       X_CO2')

# create a new reactor
gas.TDY = TDY
r = ct.IdealGasReactor(gas)
r.volume = rvol

# create a reservoir to represent the reactor immediately upstream. Note
# that the gas object is set already to the state of the upstream reactor
upstream = ct.Reservoir(gas, name='upstream')

# create a reservoir for the reactor to exhaust into. The composition of
# this reservoir is irrelevant.
downstream = ct.Reservoir(gas, name='downstream')

# Add the reacting surface to the reactor. The area is set to the desired
# catalyst area in the reactor.
rsurf = ct.ReactorSurface(surf, r, A=cat_area)

# The mass flow rate into the reactor will be fixed by using a
# MassFlowController object.
# mass_flow_rate = velocity * gas.density_mass * area  # kg/s
# mass_flow_rate = flow_rate * gas.density_mass
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

# We need an outlet to the downstream reservoir. This will determine the
# pressure in the reactor. The value of K will only affect the transient
# pressure difference.
v = ct.PressureController(r, downstream, master=m, K=1e-5)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 12

# set relative and absolute tolerances on the simulation
sim.rtol = rtol
sim.atol = atol

gas_names = gas.species_names
surf_names = surf.species_names
gas_out = []
surf_out = []
dist_array = []
T_array = []
net_rates_of_progress = []

total_flux = {"gas": dict(), "surf": dict()}
surf_diagrams = []
rsurf.kinetics.set_multiplier(0.0)  # no surface reactions until the gauze
for n in range(N_reactors):
    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = r.thermo.TDY
    upstream.syncState()
    if n == on_catalyst:
        rsurf.kinetics.set_multiplier(1.0)
        if sens is not False:
            rsurf.kinetics.set_multiplier(1.0 + sens[0], sens[1])
    if n == off_catalyst:
        rsurf.kinetics.set_multiplier(0.0)
    sim.reinitialize()
    sim.advance(sim.time + 1e4 * residence_time)
    dist = n * reactor_len * 1.0e3  # distance in mm
    dist_array.append(dist)
    T_array.append(rsurf.kinetics.T)
    kmole_flow_rate = mass_flow_rate / gas.mean_molecular_weight  # kmol/s
    gas_out.append(1000 * 60 * kmole_flow_rate * r.kinetics.X.copy())  # molar flow rate in moles/minute
    surf_out.append(rsurf.kinetics.X.copy())
    net_rates_of_progress.append(rsurf.kinetics.net_rates_of_progress)
    # surfs.append(surf)
    # stop simulation when things are done changing, to avoid getting so many COVDES errors
    if n >= 1001:
        if np.max(abs(np.subtract(gas_out[-2], gas_out[-1]))) < 1e-15:
            break
    if n>0:
        # total_flux['gas'], total_flux['surf'] = add_fluxes(gas, surf, total_flux)
        gas_fluxes = total_flux["gas"]
        surf_fluxes = total_flux["surf"]
        for element in "HOC":
            new_diagram_gas = ct.ReactionPathDiagram(gas, element)
            new_diagram_gas.get_data() # neccessary before adding this into the other diagrams, this calls build function to build the fluxes first
            new_diagram_surf = ct.ReactionPathDiagram(surf, element)
            new_diagram_surf.get_data()
            try:
                gas_fluxes[element].add(new_diagram_gas)
            except KeyError:
                gas_fluxes[element] = new_diagram_gas
            try:
                surf_fluxes[element].add(new_diagram_surf)
            except KeyError:
                surf_fluxes[element] = new_diagram_surf
        # Now do the 'X' for just the surface
        element = "X"
        try:
            new_diagram_surf = ct.ReactionPathDiagram(surf, element)
            new_diagram_surf.get_data() # neccessary before adding this into the other diagrams, this calls build to build the fluxes first
            surf_fluxes[element].add(new_diagram_surf)
        except KeyError:
            surf_fluxes[element] = new_diagram_surf

    # make reaction diagrams
    out_dir = 'rxnpath'
    os.path.exists(out_dir) or os.makedirs(out_dir)
    elements = ['H', 'O', 'C']
    locations_of_interest = [1045]
    if sens is False:
        if n in locations_of_interest:
            # # plot the transient flux_diagrams
            # transient_flux_diagram(n, surf, out_dir, ratio)
            # save_flux_diagrams(gas, surf, total_flux, path='rxnpath', suffix=f'{ratio:.1f}_{n}')
            df = pd.DataFrame(surf.net_rates_of_progress)
            df.to_csv(f"{out_dir}/net_rates_{ratio:.1f}_{n}.csv")
        # if (not (n-1) % 100) or n == (self.number_of_reactors - 1):
        #     save_flux_data(
        #         f"flux_data_Temp_{r.T-273.15:.0f}C_Dist_{distance_mm:.1f}"
        #     )

    if verbose is True:
        if not n % 100:
            print('  {0:10f}  {1:10f}  {2:10f}  {3:10f} {4:10f} {5:10f} {6:10f}'.format(dist, *gas[
                'CH4(2)', 'O2(3)', 'H2(6)', 'CO(7)', 'H2O(5)', 'CO2(4)'].X * 1000 * 60 * kmole_flow_rate))

gas_out = np.array(gas_out)
surf_out = np.array(surf_out)
gas_names = np.array(gas_names)
surf_names = np.array(surf_names)
data_out = gas_out, surf_out, gas_names, surf_names, dist_array, T_array, i_ar, n_surf_reactions

net_rates_of_progress = np.array(net_rates_of_progress)
df = pd.DataFrame(net_rates_of_progress)

out_dir = os.path.join(out_root, 'rates_of_progress')
os.path.exists(out_dir) or os.makedirs(out_dir)
df.to_csv(f"{out_dir}/net_rates_{ratio:.1f}.csv")
print(len(dist_array))
print(f"Finished monolith simulation for CH4 and O2 concs {mol_in[0], mol_in[1]} on thread {threading.get_ident()}")

This mechanism contains 40 gas reactions and 107 surface reactions
Thread ID from threading47329697548416
Running monolith simulation with CH4 and O2 concs (0.20127795527156547, 0.16773162939297123) on thread 47329697548416
7001
Finished monolith simulation for CH4 and O2 concs (0.20127795527156547, 0.16773162939297123) on thread 47329697548416


In [6]:
total_flux['surf']['X'].get_data()

'\n\n'

In [141]:
rpd = gas_fluxes['C']
print(rpd.get_data().splitlines()[2])
new = ct.ReactionPathDiagram(gas,'C')
# rpd.add(new)
print(new.get_data().splitlines()[2])
# print(new  == ct.ReactionPathDiagram(gas,'C'))
# print(new.get_data().splitlines()[2])
rpd.add(new)
print(rpd.get_data().splitlines()[2])


CH3(10) CH4(2) 0.774637 -0.774095
CH3(10) CH4(2) 0.0860708 -0.0860105
CH3(10) CH4(2) 0.860708 -0.860105


In [10]:
new = ct.ReactionPathDiagram(surf,'C')
new.get_data()

'\n\n'